# Intro a Django Forms

Form: una forma de recolectar datos de los usuario

1. GET - Obtener datos como usuarios
2. POST - Guardar datos como usuarios

In [ ]:
python manage.py startapp forms_test

### src/forms_test/forms.py

In [ ]:
from django import forms

class SearchForm(forms.Form):
    q = forms.CharField()

### src/forms_test/urls.py

In [ ]:
from django.contrib import admin
from django.urls import path

from .views import home

urlpatterns = [
    path("admin/", admin.site.urls),
    path("", home),
]

### src/forms_test/views.py

In [ ]:
from django.shortcuts import render
from .forms import SearchForm

def home(request):
    form = SearchForm()
    return render(request, "forms.html", {"form":form})

### src/config/settings.py

In [ ]:
    "forms_test.apps.FormsTestConfig", # <-------

### src/config/urls.py

In [ ]:
    path("forms/", include("forms_test.urls")), # <-------

# Tipos de campos de entrada para el form

### src/forms_test/forms.py

In [ ]:
from django import forms

class TestForm(forms.Form):
    text_input = forms.CharField()
    bool_input = forms.BooleanField()
    int_input = forms.IntegerField()
    email_input = forms.EmailField()

### src/forms_test/views.py

In [ ]:
from django.shortcuts import render

from .forms import TestForm

def home(request)
    form = TestForm(request.POST or None)
    return render(request, "forms.html", {"form": form})

## Obtener datos de un Django Form

### src/forms_test/views.py

In [ ]:
from django.shortcuts import render

from .forms import SearchForm

def home(request):
    form = SearchForm()
    if form.is_valid():
        print(form.cleaned_data)
    return render(request, "forms.html", {"form": form})

## Validaciones básicas para el Form

### src/forms_test/forms.py

In [ ]:
from dejango import forms

class TestForms(forms.Form):
    text_input = forms.CharField()
    bool_input = forms.BooleanField()
    int_input = forms.IntegerField()
    email_input = forms.EmailField()

    def clean_int_input(self, *args, **kwargs):
        entero = self.cleaned_data.get("text_input")
        if (entero<0):
            raise.ValidatorError("El valor debe ser positivo")
        return entero

    def clean_text_input(self, *args, **kwargs):
        texto = self.cleaned_data.get("text_input")
        if(len(texto) < 30):
            raise.ValidatorError("Ingresa al menos 30 caracteres")
        return texto

## Datos iniciales en un Form
- En el Form
- En el View

### src/forms_test/forms.py
- Estas configuraciones toman segundo orden de prioridad, es posible que no sean mostradas

In [ ]:
from django import forms

class SearchForm(forms.Form):
    text_input = forms.CharField()
    bool_input = forms.BooleanField(initial=True)
    int_input = forms.IntegerField()
    email_input = forms.EmailField()
    
    def clean_int_input(self, *args, **kwargs):
        entero = self.cleaned_data.get("int_input")
        if(entero < 0):
            raise forms.ValidationError("El número no puede ser negativo")
        return entero
    
    def clean_text_input(self, *args, **kwargs):
        texto = self.cleaned_data.get("text_input")
        if(len(texto) < 30):
            raise forms.ValidationError("Utiliza al menos 30 caracteres")
        return texto

### src/forms_test/views.py
- Estas configuraciones toman primer orden de prioridad y son mostradas 

In [ ]:
from django.shortcuts import render

from .forms import SearchForm

def home(request):
    initial_data = {
        "text_input": "base text",
        # "bool_input": True,
        "int_input": 1,
        "email_input": "sample@mail.ml"
    }
    form = SearchForm(request.POST or None, initial=initial_data)
    if form.is_valid():
        print(form.cleaned_data)
    return render(request, "forms.html", {"form": form})

## Etiquetas y Widgets de Forms
- Por default la etiqueta del input será la misma que el nombre del campo en el formato
- Asignar un valor al atributo "label" sobreescribe el valor default

### src/forms_test/forms.py

In [ ]:
from django import forms
import datetime

YEARS = range(1920, datetime.date.today().year + 1)

# El usuario ve el segundo valor de la tupla
# El primer valor es el que se envía al servidor
CHOICES = [
    ("a", "Opción A"),
    ("b", "Opción B"),
    ("c", "Opción C"),
]

class SearchForm(forms.Form):
    date_input = forms.DateField(
        initial=datetime.date.today,  # Establecer la fecha actual
        widget=forms.SelectDateWidget(years=YEARS),  # Permitir hasta el año actual
        label="Fecha:"
    )
    text_input = forms.CharField(
        label="Descripción:",
        widget=forms.Textarea(attrs={"cols": 90, "rows": 3})
    )
    bool_input = forms.BooleanField(label="Asignar:", initial=True)
    int_input = forms.IntegerField(label="Cantidad:")
    email_input = forms.EmailField(label="Email:")
    options_input = forms.CharField(
        label="Selecciona una opción:",
        widget=forms.Select(choices=CHOICES)
    )
    radio_input = forms.CharField(
        label="Selecciona una opción:",
        widget=forms.RadioSelect(choices=CHOICES)
    )
    checkbox_input = forms.CharField(
        label="Selecciona las que apliquen:",
        widget=forms.CheckboxSelectMultiple(choices=CHOICES)
    )
    
    def clean_int_input(self):
        entero = self.cleaned_data.get("int_input")
        if(entero < 0):
            raise forms.ValidationError("El número no puede ser negativo")
        return entero
    
    def clean_text_input(self):
        texto = self.cleaned_data.get("text_input")
        if(len(texto) < 30):
            raise forms.ValidationError("Utiliza al menos 30 caracteres")
        return texto

## Model Form

### src/forms_test/forms.py

In [ ]:
... from .models import Product

class ProductModelForm(forms.ModelForm):
    class Meta:
        model = Product
        fields = [
            "title",
            "slug",
            "price",
        ]
        exclude = []

### src/forms_test/models.py

In [ ]:
from django.db import models
from django.db import models

class Product(models.Model):
    title = models.CharField(max_length=255)
    slug = models.SlugField(max_length=255, unique=True)
    price = models.FloatField()

### src/forms_test/admin.py

In [ ]:
from django.contrib import admin

from .models import Product

admin.site.register(Product)

## Django Formsets

### src/templates/base.html

In [ ]:
<body>
    <div class="container">
        {% block content %}
        {% endblock %}
    </div>
</body>

### src/templates/formset_view.html

In [ ]:
{% extends "base.html" %}

{% block content %}

<h1>Formset View</h1>

<form action="" method="POST">
    {% csrf_token %}

    {{ formset.management_form }}
    {% for form in formset %}
        <div>
            {{form.as_p}}
            <br/>
        </div>
    {% endfor %}

    <input type="submit" value="Guardar">
</form>

### src/forms_test/views.py

In [ ]:
from django.shortcuts import render
from django.forms import formset_factory, modelformset_factory

from .forms import SearchForm, ProductModelForm

def home(request):
    TestFormSet = formset_factory(SearchForm, extra=3)
    formset = TestFormSet(request.POST or None)
    
    for form in formset:
        print(form.data)
    context = {
        "formset": formset,
    }
    # initial_data = {
        # "text_input": "base text",
        # "bool_input": True,
        # "int_input": 1,
        # "email_input": "sample@mail.ml"
    # }
    # form = SearchForm(request.POST or None, initial=initial_data)
    # if form.is_valid():
        # print(form.cleaned_data)
    return render(request, "formset_view.html", context)

## Django ModelFormsets
- Es un grupo de ModelForms

### src/forms_test/views.py

In [ ]:
from django.forms import formset_factory, modelformset_factory
from django.shortcuts import render

from .forms import SearchForm, ProductModelForm
from .models import Product

def home(request):
    ProductModelFormSet = modelformset_factory(Product, form=ProductModelForm)
    formset = ProductModelFormSet(request.POST or None, queryset=Product.objects.all())
    
    print("formset.data")
    print(formset.data)
    print("formset.errors")
    print(formset.errors)
    
    # formset.clean()
    if formset.is_valid():
        print("ModelFormset es válido")
        formset.save()
    
    # for form in formset:
        # print(form.data)
    context = {
        "formset": formset,
    }
    # initial_data = {
        # "text_input": "base text",
        # "bool_input": True,
        # "int_input": 1,
        # "email_input": "sample@mail.ml"
    # }
    # form = SearchForm(request.POST or None, initial=initial_data)
    # if form.is_valid():
        # print(form.cleaned_data)
    return render(request, "formset_view.html", context)